# Type Hints in Functions

Example 8-1. `show_count` from messages.py without type hints

In [1]:
from __future__ import annotations

"""
Goal: type the function with the following behavior
>>> show_count(99, "bird")
'99 birds'
>>> show_count(1, "bird")
'1 bird'
>>> show_count(0, "bird")
'no birds'
"""

def show_count(count: int, word: str) -> str:
    if count == 1:
        return f'1 {word}'
    count_str = str(count) if count else 'no'
    return f'{count_str} {word}s'

In [2]:
! pip install mypy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip


In [3]:
! mypy ./messages/no_hints/messages.py

messages/no_hints/messages.py:14: error: Function is missing a return type annotation  [no-untyped-def]
messages/no_hints/messages.py:14: error: Function is missing a type annotation for one or more arguments  [no-untyped-def]
Found 2 errors in 1 file (checked 1 source file)


Why do we see success here if mypy is a type checker and this function is not typed?

   Example 8-2. `messages_test.py` without type hints
   

In [4]:
from pytest import mark

@mark.parametrize('qty, expected', [
    (1, '1 part'),
    (2, '2 parts'),
])
def test_show_count(qty, expected):
    got = show_count(qty, 'part')
    assert got == expected

def test_show_count_zero():
    got = show_count(0, 'part')
    assert got == 'no parts'


In [5]:
! mypy --disallow-untyped-defs messages/no_hints/messages_test.py

messages/no_hints/messages_test.py:9: error: Function is missing a type annotation  [no-untyped-def]
messages/no_hints/messages_test.py:13: error: Function is missing a return type annotation  [no-untyped-def]
messages/no_hints/messages_test.py:13: note: Use "-> None" if function does not return a value
Found 2 errors in 1 file (checked 1 source file)


Now that we know where our untyped functions are, we can annotate them.

Outside the context of book club, you'll probably never use the command line arguments for mypy. You'll use a configuration file.

```
[mypy]
python_version = 3.9
warn_unused_configs = True
disallow_incomplete_defs = True
```

The `show_count` function only works with regular nouns. Let's add an optional parameter for the plural form to be given. 
```python
>>> show_count(3, 'mouse', 'mice')
'3 mice'
```

### A Default Parameter Value

Example 8-3. `show_count` from hints_2/messages.py with an optional parameter

In [6]:
def show_count(count: int, singular: str, plural: str = '') -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'

show_count(2, "mouse", plural="mice")

'2 mice'

In [7]:
! mypy messages/hints_2/messages.py

Success: no issues found in 1 source file


The next cell has an error. Where is it?

In [8]:

def hex2rgb(color=str) -> tuple[int, int, int]:
    ...
    pass

## Using None as Default

In [9]:
# how can we add typing for `plural`?
from __future__ import annotations


def show_count(count: int, singular: str, plural: str | None = None):
    pass

## Types are Defined by Supported Operations

In [10]:
def double(x):
    return x * 2

double(['1', '2', '3'])

['1', '2', '3', '1', '2', '3']

In [11]:
from collections import abc

def double(x: abc.Sequence):
    return x * 2

A type checker will reject that code because `abc.Sequence` does not implement or inherit the `__mul__` method.

But at run time, the code works without errors.

In [12]:
double([1])

[1, 1]

Two different views of types are interfering with each other.

- Duck typing
- Nominal typing

Example 8-4. `birds.py`

In [13]:
class Bird:
    pass

class Duck(Bird):  # <1>
    def quack(self):
        print('Quack!')

def alert(birdie):  # <2>
    birdie.quack()

def alert_duck(birdie: Duck) -> None:  # <3>
    birdie.quack()

def alert_bird(birdie: Bird) -> None:  # <4>
    birdie.quack()


In [14]:
! mypy birds/birds.py

birds/birds.py:15: error: "Bird" has no attribute "quack"  [attr-defined]
Found 1 error in 1 file (checked 1 source file)


Example 8-5. `daffy.py`

In [15]:
daffy = Duck()
alert(daffy)       # <1>
alert_duck(daffy)  # <2>
alert_bird(daffy)

Quack!
Quack!
Quack!


In [16]:
! mypy birds/daffy.py

Success: no issues found in 1 source file


No problem with `daffy.py`. Only with `birds.py`.

Example 8-6. `woody.py`

In [17]:
woody = Bird()
alert(woody)
alert_duck(woody)
alert_bird(woody)

AttributeError: 'Bird' object has no attribute 'quack'

In [18]:
! mypy birds/woody.py

birds/woody.py:5: error: Argument 1 to "alert_duck" has incompatible type "Bird"; expected "Duck"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


Example 8-7. Runtime errors and how Mypy could have helped

In [19]:
# Mypy could not detect this error because there are no type hints in `alert`
woody = Bird()
alert(woody)

AttributeError: 'Bird' object has no attribute 'quack'

In [20]:
# Mypy reported the problem before runtime!
alert_duck(woody)

AttributeError: 'Bird' object has no attribute 'quack'

In [21]:
# Mypy has been telling us since example 8-4 that the body of the `alert_bird` function is wrong: "Bird" has no attribute "quack"

alert_bird(woody)

AttributeError: 'Bird' object has no attribute 'quack'

## Types Usable in Annotations

Be conservative in what you send, be liberal with what you accept.
-- Postel's law, aka the Robustness Principle

Example 8-15. `replacer.py`

In [22]:
from collections.abc import Iterable

FromTo = tuple[str, str]  # type alias to make signature more readable

def zip_replace(text: str, changes: Iterable[FromTo]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)
    return text

In [23]:
zip_replace("Hi there", [("there", "Jamie")])

'Hi Jamie'

In [24]:
zip_replace("Hi there", {("there", "Jamie")})

'Hi Jamie'

In [25]:
tup = (1, 2)
tup[0] = 2

TypeError: 'tuple' object does not support item assignment

## Parameterized Generics and TypeVar

Example 8-16. `sample.py`

In [26]:

from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

T = TypeVar('T')

def sample(population: Sequence[T], size: int) -> list[T]:
    if size < 1:
        raise ValueError('size must be >= 1')
    result = list(population)
    shuffle(result)
    return result[:size]

In [28]:
sample(["horse", "fish", "dolphin", "cow", "cat"], 2)

['cow', 2]

In [29]:
sample("all the letters in the string", 2)

['i', 't']

Example 8-17. mode that operates on float and subtypes

In [34]:
from collections import Counter
from collections.abc import Iterable

def mode(data: Iterable[float]) -> float:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError("no mode for empty data")
    return pairs[0][0]

In [35]:
mode([1, 1, 1, 2, 2, 3])

1

Let's improve the function using TypeVar

In [36]:
from collections.abc import Iterable
from typing import TypeVar

T = TypeVar('T')

def mode(data: Iterable[T]) -> T:
    ...
    pass

The problem: we can't _really_ accept any value in the iterable. Since we're using counter, we can only accept hashable types. 

The solution: Restricted TypeVar

In [37]:
from decimal import Decimal
from fractions import Fraction

NumberT = TypeVar('NumberT', float, Decimal, Fraction)

def mode(data: Iterable[NumberT]) -> NumberT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError("no mode for empty data")
    return pairs[0][0]

In [38]:
mode(["a", "a", "a", "b"])

'a'

What if we wanted to include strings too? We could add this type, but then `NumberT` is poorly named. We can do better with Bounded TypeVar.

In [40]:
from collections.abc import Iterable, Hashable

def mode(data: Iterable[Hashable]) -> Hashable:
    pass

This return type is not very useful. 

In [45]:
# the parameter may be any subtype of hashable
HashableT = TypeVar("HashableT", bound=Hashable)  

def mode(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError("no mode for empty data")
    return pairs[0][0]

mode([1, 2, 2, 2, 3])

mode([["a"], ["b"]])

TypeError: unhashable type: 'list'

"Type hints should be used whenever unit tests are worth writing." -- Bernat Gabor

## Static Protocols

Suppose you want to create a function `top(it, n)` that returns the largest `n` elements of the iterable `it`.

```python
>>> top([4, 1, 5, 2, 6, 7, 3])
[7, 6, 5]

>>> l = "mango pear apple kiwi banana".split()
>>> top(l)
['pear', 'mango', 'kiwi']

>>> l2 = [(len(s), s) for s in l]
>>> l2
[
    (5, 'mango'),
    (4, 'pear'),
    (5, 'apple'),
    (4, 'kiwi'),
    (6, 'banana'),
]
>>> top(l2, 3)
[(6, 'banana'), (5, 'mango'), (5, 'apple')]
```

Example 8-19. `top` function with an undefined `T` type parameter

In [47]:
def top(series: Iterable[T], length: int) -> list[T]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

top([3,2,1], 1)

[3]

How can we constrain T? `sorted` accepts `Iterable[Any]`, but that's because it accepts an optional argument for a function that computes a sort key.

What happens if you give `sorted` a list of plain objects?

In [48]:
l = [object() for _ in range(4)]
l

In [49]:
sorted(l)

TypeError: '<' not supported between instances of 'object' and 'object'

Is it all it takes to just implement less than?

In [50]:
class Spam:
    def __init__(self, n):
        self.n = n

    def __lt__(self, other):
        return self.n < other.n

    def __repr__(self):
        return f'Spam({self.n})'

l = [Spam(n) for n in range(5, 0, -1)]
l

[Spam(5), Spam(4), Spam(3), Spam(2), Spam(1)]

In [51]:
sorted(l)

[Spam(1), Spam(2), Spam(3), Spam(4), Spam(5)]

So the `T` in Example 8-19 should be limited to types that implement `__lt__`.

There is no suitable type in `typing` or `abc`, so we need to create one.

Example 8-20. `comparable.py`: definition of a `SupportsLessThanProtocol` type

In [53]:
from typing import Protocol, Any

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool:
        ...  # the body of protocol has ... literally!


A type `T` is _consistent-with_ a protocol `P` if `T` implements all the methods defined in `P`, with matching type signatures.

Example 8-21. `top.py`: definition of the `top` function using a `TypeVar`
with `bound=SupportsLessThan`

In [54]:
from collections.abc import Iterable
from typing import TypeVar


LT = TypeVar('LT', bound=SupportsLessThan)

def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

Example 8-22. `top_test.py`: partial listing of the test suite for `top`

In [57]:
! mypy ./comparable/top.py

Success: no issues found in 1 source file


This is `static duck typing`. The nominal type of `series` doesn't matter, as long as it implements the `__lt__` method.

## Callable

Functions can be annotated like this
```python
Callable[[ParamType1, ParamType2], ReturnType]
```

### Variance in CallableTypes

Imagine a temperature control system with a simple `update` function.

Example 8-24. Illustrating Variance

In [63]:
from typing import Callable

def update(
        probe: Callable[[], float],
        display: Callable[[float], None]
    ) -> None:
    temperature = probe()
    # imagine lots of control code here
    display(temperature)

def probe_ok() -> int:  # <4>
    return 42

def display_wrong(temperature: int) -> None:  # <5>
    print(hex(temperature))

In [64]:
update(probe_ok, display_wrong)

0x2a


In [60]:
def display_ok(temperature: complex) -> None:  # <7>
    print(temperature)

update(probe_ok, display_ok)  # OK  # <8>

42


# Summary

> I wouldn't like a version of Python where I was morally obligated to add type hints all the time. I really do think that type hints have their place but there are also plenty of times that it's not worth it, and it's so wonderful that you can choose to use them.
> -- Guido van Rossum